# Introduction to Python :: Session 6

Brent Nef  
September 26, 2018

* <https://github.com/n3f/intro-python/tree/session06>
* <https://github.com/n3f/pcc-django>

## Jupyter on home PC

* install miniconda
* `conda create -n <some-name> python jupyter git`
* `activate <some-name>` or `conda activate <some-name>`
* `git clone https://github.com/n3f/intro-python.git`
* Check out the session you want to see (e.g. session06): `git checkout session06`
* `jupyter notebook`

# What we're building

> We’ll write a web app called Learning Log that allows users to
log the topics they’re interested in and to make journal entries
as they learn about each topic. The Learning Log home page
should describe the site and invite users to either register or log
in. Once logged in, a user should be able to create new topics,
add new entries, and read and edit existing entries.

# Create environment

```sh
conda create -n djangoEnv python django ipython
# ... stuff ...
conda activate djangoEnv
```

* You can optionally pass in the versions that you want to install if you needed to for example reproduce a bug (e.g. python==3.6.1, django==1.8.10)
* The `pipenv` project helps to create reproducible application environments, but I haven't had the greatest experience with it

## More Environments

* If you are using pip you put your packages into `requirements.txt`
* Conda uses any file, but I like `environment.yml`
* If you had the following in `environment.yml` you would create with `conda create --file environment.yml`

  ```yml
  name: djangoEnv
  dependencies:
  - python>=3.5
  - ipython
  - django
  ```

# Getting started with Django

## Create project

```sh
django-admin startproject learning_log . # Don't miss the dot!
```

## Created files

```
(djangoEnv) C:\Users\brent.nef\Documents\intro-python\django>dir /b
learning_log
manage.py

(djangoEnv) C:\Users\brent.nef\Documents\intro-python\django>dir /b learning_log
settings.py
urls.py
wsgi.py
__init__.py
```

## Create the database

```sh
python manage.py migrate
```

* Django manages your database changes for you (you can change this or use a different database or a different ORM, but you *are* using django for a reason...
* Database changes are called migrations.  This first migration makes the database match the current project structure

```
(djangoEnv) C:\Users\brent.nef\Documents\intro-python\django>python manage.py migrate
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying sessions.0001_initial... OK

(djangoEnv) C:\Users\brent.nef\Documents\intro-python\django>dir /b
db.sqlite3
learning_log
manage.py
```

## Running django

```sh
python manage.py runserver
```

* OK, we can go home now right?

# Creating an app

```sh
python manage.py startapp learning_logs
```

## Defining models

* The objects that you want your application to use need to be defined as django *models*
* Use the django `Model` as a base class, lets django know how to interface with your object
   * by following a couple rules here you get a full database interface without having to manage much

In [ ]:
class Topic(models.Model):
    """A topic the user is learning about"""
    text = models.CharField(max_length=200)
    date_added = models.DateTimeField(auto_now_add=True)
    
    def __str__(self):
        """Return a string representation of the model."""
        return self.text

## Model reference & Django documentation

* Field types: <https://docs.djangoproject.com/en/2.1/ref/models/fields/#field-types>

## Activating models

* modify `settings.py` and add our app to the list of `INSTALLED_APPS`
* Steps you always do after modifying `models.py`:
   1. make migrations: `python manage.py makemigrations learning_logs`
   1. apply migrations: *same as before: `python manage.py migrate`*

# Django Admin

* sweet management interface you get for free
* first need to create a superuser: `python manage.py createsuperuser`
* Add your models in `admin.py`:

In [ ]:
from django.contrib import admin
from .models import Topic

# Register your models here.
admin.site.register(Topic)

##### Go check it out

## Add Entry model

In [ ]:
class Entry(models.Model):
    """Something specific learned about a topic"""
    topic = models.ForeignKey(Topic, on_delete=models.SET_NULL, null=True)
    text = models.TextField()
    date_added = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        verbose_name_plural = 'entries'
    
    def __str__(self):
        """Return a string representation of the model."""
        return self.text[:50] + "..."

# Django shell

* REPL interface to django
* Makes it easy to query elements, inspect data, and try code that you want to use
* `python manage.py shell`
   * `Topic.objects.all()`, `Topic.objects.get(id=1)`, `t.entry_set.all()`

# Creating pages

1. Define the URL
1. Write the view
1. Write the template

## Defining the URL

* Create a pattern for a url that you want to respond to and point to a *view* (this is just a function)
* URLs are defined in the `urls.py` files:
   * Add `path(r'', include('learning_logs.urls'),` to the project `urls.py` *(You also have to add the `include` function to the `django.urls` import command)*

In [ ]:
"""Defines URL patterns for learning_logs."""
from django.conf.urls import url
from . import views

app_name = 'learning_logs'
urlpatterns = [
    # Home page
    url(r'^$', views.index, name='index'),
]

## Writing a view

In [ ]:
def index(request):
    """The home page for Learning Log"""
    return render(request, 'learning_logs/index.html')

## Writing a template

* Create a folder structure so that: `<djangoroot>/learning_logs/templates/learning_logs`
   * `templates` is a *well known location*, but the other `learning_logs` folder makes in unambiguous which application your template is from, and is a best practice

### index.html
```html
<p>Learning Log</p>
<p>Learning Log helps you keep track of your learning, for any topic you're
learning about.</p>
```

### Template inheritance

* Many of your sites pages will probably look similar.  Templates provide inheritance allowing you to build off of other previously created templates

#### `base.html`

```html
<p>
  <a href="{% url 'learning_logs:index' %}">Learning Log</a>
</p>
{% block content %}{% endblock content %}
```

#### `index.html`
```html
{% extends "learning_logs/base.html" %}
{% block content %}
<p>Learning Log helps you keep track of your learning, for any topic you're
learning about.</p>
{% endblock content %}
```

### Topics pages

1. List all topics
1. Display entries for a topic

#### List all topics

* Update our urls: `url(r'^topics/$', views.topics, name='topics'),`

In [ ]:
def topics(request):
    """Show all topics."""
    topics = Topic.objects.order_by('date_added')
    context = {'topics': topics}
    return render(request, 'learning_logs/topics.html', context)

#### `topics.html`

```html
{% extends "learning_logs/base.html" %}
{% block content %}
<p>Topics</p>
<ul>
{% for topic in topics %}
  <li>{{ topic }}</li>
{% empty %}
  <li>No topics have been added yet.</li>
{% endfor %}
</ul>
{% endblock content %}
```

### List single topic

* Update url: `url(r'^topics/(?P<topic_id>\d+)/$', views.topic, name='topic'),`

In [ ]:
def topic(request, topic_id):
    """Show a single topic and all its entries."""
    topic = Topic.objects.get(id=topic_id)
    entries = topic.entry_set.order_by('-date_added')
    context = {'topic': topic, 'entries': entries}
    return render(request, 'learning_logs/topic.html', context)

#### `topic.html`
```html
{% extends 'learning_logs/base.html' %}
{% block content %}
<p>Topic: {{ topic }}</p>
<p>Entries:</p>
<ul>
{% for entry in entries %}
  <li>
  <p>{{ entry.date_added|date:'M d, Y H:i' }}</p>
  <p>{{ entry.text|linebreaks }}</p>
  </li>
{% empty %}
  <li>
  There are no entries for this topic yet.
  </li>
{% endfor %}
</ul>
{% endblock content %}
```

#### Add link in `topics.html`
```html
<li>
  <a href="{% url 'learning_logs:topic' topic.id %}">{{ topic }}</a>
</li>
```

# User Accounts

* Possible to grant access of the admin site to other users (where we can add topics/entries)
* Possible to create specific views to do as well
   * Create form using `ModelForm` class

## Topics `ModelForm`

### `forms.py`

In [ ]:
from django import forms
from .models import Topic

class TopicForm(forms.ModelForm):
    class Meta:
        model = Topic
        fields = ['text']
        labels = {'text': ''}

* url: `url(r'^new_topic/$', views.new_topic, name='new_topic'),`

In [ ]:
from django.http import HttpResponseRedirect
from django.urls import reverse

def new_topic(request):
    """Add a new topic."""
    if request.method != 'POST':
        # No data submitted; create a blank form.
        form = TopicForm()
    else:
        # POST data submitted; process data.
        form = TopicForm(request.POST)
    if form.is_valid():
        form.save()
        return HttpResponseRedirect(reverse('learning_logs:topics'))
    context = {'form': form}
    return render(request, 'learning_logs/new_topic.html', context)

#### `new_topic.html`

```html
{% extends "learning_logs/base.html" %}

{% block content %}
<p>Add a new topic:</p>

<form action="{% url 'learning_logs:new_topic' %}" method='post'>
  {% csrf_token %}
  {{ form.as_p }}
  <button name="submit">add topic</button>
</form>

{% endblock content %}
```

#### `topics.html`

`<a href="{% url 'learning_logs:new_topic' %}">Add a new topic:</a>`

## Entries

* Now I'm going to cheat a little and fast-forward
* Follow similar pattern for viewing entries (`new_entry.html`)
* Add additional url/view/template for editing entries (`edit_entry.html`)

# Creating user accounts

* We'll create a new app for managing users: `python manage.py startapp users`
   * Add `users` to `settings.py`
   * Add `users.urls` to `urls.py`
* Create a login page

## Login page

1. `urls.py` login url uses django auth login view
1. login template: `login.html`, add link to `base.html`

## Logout view

1. `urls.py` logout url uses django auth logout view
1. `base.html` 👉 link to logout

## New users & registration

1. `urls.py` add the register url
1. `views.py` register view
1. `register.html`

## Restricting access

1. `views.py`: `login_required` decorator
1. Modify `settings.py` with `LOGIN_URL`

## Tying data to a user

1. Change model adding owner to `Topic`
1. Assign current data to a user
   1. Get the id of the user that should own data
   1. `python manage.py makemigrations learning_logs`
   1. Migrate the data
1. Restrict access to owner
   1. Only show topics owned by user: `views.py:topics`
   1. check that the user owns topic or `edit_entry`: `views.py:topic/edit_entry`
   1. Associate new topics with current user: `views.py:new_topic`

# What we did

* Built a functional web application
  * login/logout/data checks
  * CRU~~D~~ operations on business data

# What's left

* Styling your app
* Deploying

# Some thoughts

* Check out django-rest-framework
* Other django plugins
   * django-debug-toolbar
   * django-bootstrap   

# The End 😢

* Thanks for attending!